In [33]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

In [42]:
# grofers 
def get_data_grofers(ptype, pageNo):
    headers = {"User-Agent":"Mozilla/5.0 (Windows NT 10.0; Win64;     x64; rv:66.0) Gecko/20100101 Firefox/66.0", \
               "Accept-Encoding":"gzip, deflate",     "Accept":"text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8", "DNT":"1","Connection":"close", "Upgrade-Insecure-Requests":"1"}
    
    #print("https://grofers.com/s/?q="+ptype)
    r = requests.get("https://grofers.com/s/?q="+ptype, headers=headers)
    
    content = r.content
    soup = BeautifulSoup(content)
    #print(soup.prettify())
    
    #x=soup.find('div', attrs={'class':'plp-product'})
    #print(x)
    
    p_product_df = pd.DataFrame(columns=['Type','Name','Qty','MRP','Price'])
    
    for d in soup.findAll('div', attrs={'class':'plp-product'}):
        name = d.find('div', attrs={'class':'plp-product__name--box'})
        price = d.find('span', attrs={'class':'plp-product__price--new'})
        mrp = d.find('div', attrs={'class':'plp-product__price--old display--inline-block@mobile'})
        qty = d.find('div', attrs={'class':'plp-product__quantity'})
    
        #print(rating.text)   
        if name is not None:
            final_name = name.text
        else:
            final_name = "unknown-product"
       
        if price is not None:
            final_price = price.text
            final_price = final_price[1:]
        else:
            final_price = '0'
        
        if mrp is not None:
            final_mrp = mrp.text
            final_mrp = final_mrp[1:]
        else:
            final_mrp = final_price      

        if qty is not None:
            final_qty = qty.text
        else:
            final_qty = '-1'
        
        #print(ptype,final_name, final_price, final_qty)
        #product_df.append({'Name': 'acer','Price':'1000$','Rating':'4 out of 5'}, ignore_index=True)
        p_product_df = p_product_df.append({'Type':ptype,'Name': final_name,'MRP':final_mrp,'Price':final_price,'Qty':final_qty}, ignore_index=True)
        
    return p_product_df

In [43]:
data_df = pd.DataFrame(columns=['Type','Name','Qty','MRP','Price'])
no_pages = 1

ptype_list = ['oil', 'cereal','rice','dal','biscuit','juice','toothpaste','soap','chocolates','shampoo','drinks']

for x in ptype_list:
    for i in range(no_pages):
        data_df = data_df.append(get_data_grofers(x,i),ignore_index=True)

In [44]:
data_df.to_csv('grofer_products.csv')
data_df.head()

,Type,Name,Qty,MRP,Price
0,oil,Fortune Soya Health Refined Soyabean Oil (Pouch),1 l,130,110
1,oil,Grofers Mother's Choice Refined Soyabean Oil (...,1 l,120,105
2,oil,Fortune Premium Kachi Ghani Pure Mustard Oil (...,1 l,155,125
3,oil,Grofers Mother's Choice Kachi Ghani Mustard Oi...,1 l,145,114
4,oil,Grofers Mother's Choice Kachi Ghani Mustard Oi...,1 l,141,108
